# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [71]:
import requests
import time
import re
import functools

In [72]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [73]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    # Тут я с Фибоначчи слегка запуталась и бенчмарком, сначала написала такой код, но время показывалось для каждого рекурсивного вызова
    # Так что пришлось костылить :)

    # @functools.wraps(func)
    # def wrapper(*args, **kwargs):
    #     start_time = time.time()
    #     result = func(*args, **kwargs)
    #     end_time = time.time()
    #     print(f"Время выполнения функции {func.__name__}: {end_time - start_time} секунд")
    #     return result
    # return wrapper


    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if not hasattr(func, 'called'):
            func.called = True
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            print(f"Время выполнения функции {func.__name__}: {end_time - start_time} секунд")
        else:
            result = func(*args, **kwargs)
        return result
    return wrapper

## Задание 2

In [74]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Функция вызвана с параметрами:\n{args}, {kwargs}")
        result = func(*args, **kwargs)
        return result
    return wrapper

## Задание 3

In [75]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        print(f"Функция была вызвана: {wrapper.count} раз")
        result = func(*args, **kwargs)
        return result
    wrapper.count = 0
    return wrapper

## Задание 4

In [76]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    cache = {}

    @functools.wraps(func)
    def fmemo(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args] 

    fmemo.cache = cache
    return fmemo

## Тестирование

In [77]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция была вызвана: 1 раз
Функция вызвана с параметрами:
('whole',), {}
Время выполнения функции word_count: 2.9616971015930176 секунд
Cлово whole встречается 176 раз


In [78]:
@benchmark
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [79]:
# измеряем время выполнения
fib(30)

Время выполнения функции fib: 0.3323981761932373 секунд


832040

In [80]:
@benchmark
@memo
def fib_memo(n):
    if n < 2:
        return n
    return fib_memo(n-2) + fib_memo(n-1)

In [81]:
# измеряем время выполнения
fib_memo(30)

Время выполнения функции fib_memo: 1.9073486328125e-05 секунд


832040